# Задание 1


### Целью этого задания является знакомство со стандартными контейнерами и некторыми функциями из стандартных библиотек для машинного обучения.

Напишите наивный байесовский классификатор и сравните его с реализацией NaiveBayesClassifier из библиотеки nltk.

Написанный вами классификатор должен обладать следубщими свойствами:
<ul>
<li>В предложенном интерфейсе класса должны быть реализованы все методы и все поля. Для их хранения предподсчитанных данных рекомендуется использовать контейнеры Counter или defaultdict из библиотеки collections. Для предсказания категории рекомендуется использовать numpy.</li>
<li>Должна использоваться модель, предложенная в теории.</li>
<li>Точность предсказаний не менее <b>0.9</b>!</li>
<li>После реализации класса протестируйте его с помощью кроссвалидации с k=10. Рекомендуется использовать класс KFold из библиотеки sklearn.</li>
<li>Постройте постройте диаграмму размаха для классификаторов (своего и из библиотеки).</li>
</ul>

### Теория

Теория находится в файле problems1-theory.pdf

# Решение

In [262]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from nltk import NaiveBayesClassifier
from matplotlib import pyplot as plt
from tqdm import tqdm
import operator
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'size']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Прочитайте данные из файла

In [14]:
data_path = "ham-spam.csv"

In [249]:
x_message = []
y_target = []
with open(data_path, mode='r') as f:
    next(f)
    for line in f:
        target, msg = line.split(',', maxsplit=1)
        x_message.append(msg.lower())
        y_target.append(target.lower())

### Реализуйте все методы в классе NaiveBayes

In [291]:
class NaiveBayes(object):
    """
    Наивный байесовский классификатор.
    Для каждого входного сообщения слово учитывается один раз при расчете итоговой вероятности.

    Parameters
    ----------
    category_priors : default | None, optional, default None
        Априорные вероятности категорий.
        Если None, то классификатор должен сам их вычислить.

    weight : float, optional, default 1
        Вес одного слова в формуле взвешенной вероятности

    supposed_prob : float, optional, default 0.5
        Предполагаемая вероятность слова в категории
    """

    def __init__(self, category_priors=None, weight=1, supposed_prob=0.5):
        self.category_priors = category_priors
        self.weight = weight
        self.supposed_prob = supposed_prob

        # Количество отдельных слов в заданной категории
        self.feature_category_counts = {}
        self.sum_feature_counts = {}
        
        #Количество отдельных слов в статьях в данной категории
        self.feature_category_article_counts = {}
        self.sum_feature_category_article_counts = {}

        # Количество всех документов в данной категории
        self.category_doc_counts = {}
        self.all_docs_count = 0
        
        # Количество встреч слова во всех сообщениях
        self.feature_counts = {}

    def fit(self, x_train, y_train):
        """
        Производит обучение наивного байесовского классификатора.

        Parameters
        ----------
        text : list of list of str | list of str | str
            Входной текст описывается строкой, которую будет токенизирована по пробелу.
            Если строка не токенизирована, то текст должен быть токенизирован.
            Может быть передано несколько сообщений, которые будут токенезированы, если необходимо.

        y_train : list of str
            содержит список меток (названий категорий) для сообщений из x_train

        Returns
        -------
        self : object
            Returns self
        """
        # Подсчитываем количество категорий, документов и слов в каждой категории
        # и количество встреч слова во всех сообщениях
        for i in range(len(y_train)):            
            if y_train[i] not in self.feature_category_counts:
                self.feature_category_counts[y_train[i]] = {}
                
            if y_train[i] not in self.sum_feature_counts:
                self.sum_feature_counts[y_train[i]] = 0
                
            if y_train[i] not in self.feature_category_article_counts:
                self.feature_category_article_counts[y_train[i]] = {}
            
            if y_train[i] not in self.sum_feature_category_article_counts:
                self.sum_feature_category_article_counts[y_train[i]] = 0
            
            if y_train[i] not in self.category_doc_counts:
                self.category_doc_counts[y_train[i]] = 0                  
                    
            for line in x_train[i]:
                self.all_docs_count += 1                
                self.category_doc_counts[y_train[i]] += 1
                line = line.split(' ')
                
                for word in set(line):
                    if word not in self.feature_category_article_counts[y_train[i]]:
                        self.feature_category_article_counts[y_train[i]][word] = 0
                    if word not in self.feature_counts:
                        self.feature_counts[word] = 0 
                    self.feature_counts[word] += 1
                    self.feature_category_article_counts[y_train[i]][word] += 1
                    self.sum_feature_category_article_counts[y_train[i]] += 1
                
                for word in line:
                    if word not in self.feature_category_counts[y_train[i]]:
                        self.feature_category_counts[y_train[i]][word] = 0
                    self.feature_category_counts[y_train[i]][word] += 1
                    self.sum_feature_counts[y_train[i]] += 1
                     
        return self
    
    def predict(self, text):
        """
        Предсказывает метки категорий для text.

        Parameters
        ----------
        text : list of list of str | list of str | str
            Входной текст описывается строкой, которую будет токенизирована по пробелу.
            Если строка не токенизирована, то текст должен быть токенизирован.
            Может быть передано несколько сообщений, которые будут токенезированы, если необходимо.

        Returns
        -------
        categories : list of str
            Возвращает названия категорий для text.
        """
        categories = []
        for line in text:
            probs = self.get_probs(line)
            categories.append(self.get_categories()[np.argmax(probs)])
        return categories

    def score(self, text, labels):
        return np.sum(np.array(self.predict(text)) == np.array(labels))/len(labels)

    def get_probs(self, text):
        """
        Считает вероятности принадлежности текста (text) к каждой из категорий

        Parameters
        ----------
        text : list of str | str
            Входной текст описывается строкой, которую будет токенизирована по пробелу.
            Если строка не токенизирована, то текст должен быть токенизирован.

        Returns
        -------
        probs : list of float
            Возвращает вероятности probs всех категорий для текста text
            в порядке их следования в self.category_doc_counts.
        """
        if not isinstance(text, list):
            text= text.split(' ')
        probs = []        
        for category in self.category_doc_counts.keys():
            probs.append(self.get_category_prob(category, text))

        return probs

    def get_category_prob(self, cat, text):
        """
        Считает логарифм вероятность принадлежности сообщения text к категории cat.

        Parameters
        ----------
        cat : str
            Название категории.

        text : list of str
            Список из слов.

        Returns
        -------
        log_prob : float
            Возвращает логарифм вероятности категории cat для текста text.
        """
        prob = 0
        for word in text:
            prob += math.log(self.get_weighted_feature_prob(cat, word))
        prob += math.log(self.category_doc_counts[cat]/self.all_docs_count)

        return prob
    
    def get_weighted_feature_prob(self, cat, feature):
        p_w_s = 0
        total = 0
        if feature in self.feature_counts:
            total = self.feature_counts[feature]
        if feature in self.feature_category_article_counts[cat]:
            p_w_s = self.feature_category_article_counts[cat][feature]/self.sum_feature_category_article_counts[cat]
        p_a_w = self.supposed_prob
        if feature in self.feature_category_counts[cat]:
            p_a_w = self.feature_category_counts[cat][feature]/self.sum_feature_counts[cat]
        prob = (self.weight*p_a_w + total*p_w_s)/(total + self.weight) 

        return prob

    def get_categories(self):
        return list(self.category_doc_counts.keys())
pass


In [292]:
method = NaiveBayes()
size = len(x_message)//2
method.fit(x_message[:size], y_target[:size])

In [293]:
print(method.score(x_message[size + 1:], y_target[size + 1:]))

0.7441217150760719


In [294]:
accuracy = []
x_message = np.array(x_message)
y_target = np.array(y_target)
kf = KFold(n_splits=5, random_state=1, shuffle=True)
for train_index, test_index in kf.split(x_message):
    x_train_part, y_train_part = x_message[train_index], y_target[train_index]
    x_test_part, y_test_part = x_message[test_index], y_target[test_index]
    clf = NaiveBayes()
    clf.fit(x_train_part, y_train_part)
    accuracy.append(clf.score(x_test_part, y_test_part))
np.mean(accuracy)

0.7829302400631084

### Сравните вашу реализацию и реализацию из библиотеки nltk

Для использования классификатора из библиотеки не забудьте предподготовить данные. Для подсчета точности этого классификатора можете использовать accuracy_score из метрик sklearn. Для подсчета точности предсказаний вашего классификатора используйте функцию score, которую вы опишете.

In [ ]:
# Предобработка данных для классификатора nltk, если требуется
<your code here>

In [ ]:
# Используйте процедуру KFold для проверки качества классификаторов
<your code here>

### Постройте графики размаха для двух классификаторов на одной фигуре.

Рекомендуется использовать встроенные функции построения графиков в pandas.

In [ ]:
<your code here>